In [13]:
import os
import pandas as pd
import plotly
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly as pys
import ipywidgets as widgets
from ipywidgets import Layout

import _global_scripts as gs

## Purpose

Alternative Specific Constants by Distance Bins

## Inputs

In [14]:
# set whether you want PA or OD analysis
pa_od = 'PA'
pa_od_function = 'pa'

In [15]:
import pandas as pd

# Load observed summary data and clean up
df_tdm_obs = (
    pd.read_csv(f"_data/base_observed_summary_{pa_od_function}.csv", low_memory=False)
    .sort_values(by=['Source', 'station'], ascending=True)
    .reset_index(drop=True)
)

if pa_od == 'PA':
    df_tdm_obs.drop(columns=['Alt_PA', 'Alt_Direct_PA', 'Alt_Transfer_PA'], inplace=True)

# Define paths for boarding and rider summaries
summary_paths = {
#   "TDM-Recalib":                       ("_data/E2.14/WFv920-E2.14.1_BY_2019_transit_brding_summary_node.csv",
#                                         "_data/E2.14/WFv920-E2.14.1_BY_2019_transit_rider_summary_link.csv"),
#   "TDM-Recalib-100-divisor-removed":   ("_data/E2.14/WFv920-E2.14.2_BY_2019_transit_brding_summary_node.csv",
#                                         "_data/E2.14/WFv920-E2.14.2_BY_2019_transit_rider_summary_link.csv"),
    "E2.14.4-Recalib-RF2.5":             ("_data/E2.14/WFv920-E2.14.4_BY_2019_transit_brding_summary_node.csv", # with 2nd part removed
                                          "_data/E2.14/WFv920-E2.14.4_BY_2019_transit_rider_summary_link.csv"),
    "E2.7.3-IvtTest":                    ("_data/E2.7/WFv920-E2.7.3_BY_2019_transit_brding_summary_node.csv",
                                          "_data/E2.7/WFv920-E2.7.3_BY_2019_transit_rider_summary_link.csv"),
    "E2.13.4-ASC-by-Distance":           ("_data/E2.13/WFv920-E2.13.4_BY_2019_transit_brding_summary_node.csv",
                                          "_data/E2.13/WFv920-E2.13.4_BY_2019_transit_rider_summary_link.csv"),
    "E2.13.4.3-ASC-by-Distance":         ("_data/E2.13/WFv920-E2.13.4.3_BY_2019_transit_brding_summary_node.csv",
                                          "_data/E2.13/WFv920-E2.13.4.3_BY_2019_transit_rider_summary_link.csv"),
    "E2.13.5-ASC-by-Distance-IvtTest":   ("_data/E2.13/WFv920-E2.13.5_BY_2019_transit_brding_summary_node.csv",
                                          "_data/E2.13/WFv920-E2.13.5_BY_2019_transit_rider_summary_link.csv"),
    "E2.13.5.2-ASC-by-Distance-IvtTest": ("_data/E2.13/WFv920-E2.13.5.2_BY_2019_transit_brding_summary_node.csv",
                                          "_data/E2.13/WFv920-E2.13.5.2_BY_2019_transit_rider_summary_link.csv")
}

# Read stations data
df_stations1 = gs.df_stations[['station', 'N']]

# Summarize TDM stats for each dataset
df_tdm_list = [df_tdm_obs]  # Start with observed data

for label, (boarding_path, rider_path) in summary_paths.items():
    df_tdm_list.append(gs.summarize_tdm_stats(boarding_path, rider_path, df_stations1, label, pa_od_function))

# Concatenate and clean the dataframe
df_tdm_obs_new = pd.concat(df_tdm_list, ignore_index=True)

# Further clean and finalize the dataframe
df_tdm_obs_new = (
    df_tdm_obs_new
    .round(3)
    .fillna(0)
    .drop(columns='index', errors='ignore')
    .loc[:, ~df_tdm_obs_new.columns.str.contains('^Unnamed')]
)

# Rename and display
sumStats = df_tdm_obs_new.copy()
display(df_tdm_obs_new)


,Source,station,AccessMode,Brd_PA,Brd_Direct_PA,Brd_Transfer_PA
0,OBS,01-PROVO CENTRAL STATION,drive,2283.379,0.00,0.00
1,OBS,01-PROVO CENTRAL STATION,walk,945.908,0.00,0.00
2,OBS,02-OREM CENTRAL STATION,drive,1475.024,0.00,0.00
3,OBS,02-OREM CENTRAL STATION,walk,371.804,0.00,0.00
4,OBS,03-AMERICAN FORK STATION,drive,1423.768,0.00,0.00
...,...,...,...,...,...,...
235,E2.13.5.2-ASC-by-Distance-IvtTest,11-FARMINGTON STATION,walk,196.170,135.71,60.46
236,E2.13.5.2-ASC-by-Distance-IvtTest,12-LAYTON STATION,walk,364.070,139.88,224.19
237,E2.13.5.2-ASC-by-Distance-IvtTest,13-CLEARFIELD STATION,walk,635.900,500.37,135.53
238,E2.13.5.2-ASC-by-Distance-IvtTest,14-ROY STATION,walk,400.370,361.95,38.42


In [16]:
# add a few more columns regarding percentage of boardings in relation to total boardings

# sum by source and station 
station_sum = sumStats.groupby(["Source", "station"], as_index=False).agg({
    f"Brd_{pa_od}": "sum",
    f"Brd_Direct_{pa_od}": "sum",
    f"Brd_Transfer_{pa_od}": "sum"
})

# add All accessMode
station_sum["AccessMode"] = "All"
sumStats2 = pd.concat([sumStats, station_sum], ignore_index=True) 

accessmode_sum = sumStats2.groupby(["Source", "AccessMode"], as_index=False).agg({
    f"Brd_{pa_od}": "sum",
    f"Brd_Direct_{pa_od}": "sum",
    f"Brd_Transfer_{pa_od}": "sum"
})

accessmode_sum.rename(columns={
    f"Brd_{pa_od}": f"Source_Brd_{pa_od}", 
    f"Brd_Direct_{pa_od}": f"Source_Brd_Direct_{pa_od}", 
    f"Brd_Transfer_{pa_od}": f"Source_Brd_Transfer_{pa_od}"}, inplace=True)
sumStatsP = sumStats2.merge(accessmode_sum, on=["Source", "AccessMode"], how="left")

sumStatsP[f"Brd_{pa_od}_Perc"]          = sumStatsP[f"Brd_{pa_od}"] / sumStatsP[f"Source_Brd_{pa_od}"]
sumStatsP[f"Brd_Direct_{pa_od}_Perc"]   = sumStatsP[f"Brd_Direct_{pa_od}"] / sumStatsP[f"Source_Brd_Direct_{pa_od}"]
sumStatsP[f"Brd_Transfer_{pa_od}_Perc"] = sumStatsP[f"Brd_Transfer_{pa_od}"] / sumStatsP[f"Source_Brd_Transfer_{pa_od}"]
sumStatsP


,Source,station,AccessMode,Brd_PA,Brd_Direct_PA,Brd_Transfer_PA,Source_Brd_PA,Source_Brd_Direct_PA,Source_Brd_Transfer_PA,Brd_PA_Perc,Brd_Direct_PA_Perc,Brd_Transfer_PA_Perc
0,OBS,01-PROVO CENTRAL STATION,drive,2283.379,0.00,0.00,14955.118,0.00,0.00,0.152682,NaN,NaN
1,OBS,01-PROVO CENTRAL STATION,walk,945.908,0.00,0.00,5672.892,0.00,0.00,0.166742,NaN,NaN
2,OBS,02-OREM CENTRAL STATION,drive,1475.024,0.00,0.00,14955.118,0.00,0.00,0.098630,NaN,NaN
3,OBS,02-OREM CENTRAL STATION,walk,371.804,0.00,0.00,5672.892,0.00,0.00,0.065540,NaN,NaN
4,OBS,03-AMERICAN FORK STATION,drive,1423.768,0.00,0.00,14955.118,0.00,0.00,0.095203,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
355,TDM,11-FARMINGTON STATION,All,581.110,539.35,41.76,18575.280,14866.16,3709.12,0.031284,0.036280,0.011259
356,TDM,12-LAYTON STATION,All,1202.070,1004.67,197.40,18575.280,14866.16,3709.12,0.064713,0.067581,0.053220
357,TDM,13-CLEARFIELD STATION,All,1259.980,1155.01,104.97,18575.280,14866.16,3709.12,0.067831,0.077694,0.028301
358,TDM,14-ROY STATION,All,1408.620,1369.71,38.91,18575.280,14866.16,3709.12,0.075833,0.092136,0.010490


In [17]:
def plotit(variable, access_mode):
    output.clear_output()  # Clear previous output before displaying new content
    global firstTime
    if firstTime:
    
        filtered_data = sumStatsP[sumStatsP['AccessMode'] == access_mode]
            
        # Create histogram
        fig = px.histogram(
            filtered_data, 
            x="station", 
            y=variable, 
            text_auto='.2s',
            color='Source', 
            barmode='group',
            height=400
        )
        fig.update_layout(
            xaxis_title="Station Name",
            yaxis_title=str(variable),
            legend_title="Model Version"
        )
        
        # Display the plot
        fig.show()
    
    else:
        firstTime = True

In [19]:
lstValues = list([
    f'Brd_{pa_od}',
    f'Brd_Direct_{pa_od}',
    f'Brd_Transfer_{pa_od}',
    f'Brd_{pa_od}_Perc',
    f'Brd_Direct_{pa_od}_Perc',
    f'Brd_Transfer_{pa_od}_Perc'])
accessModeOptions = ['drive', 'walk', 'All']

selectValues = widgets.Select(options=lstValues, value=(f'Brd_{pa_od}' ), description = 'Select Variable')
selectAccessMode = widgets.Dropdown(options=accessModeOptions, value='All', description='Access Mode')

# Set up a global variable to track whether the widgets have been changed
firstTime = False

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([selectValues, selectAccessMode])

# create interactive widget
interactive_output = widgets.interactive_output(plotit, {'variable':selectValues, 'access_mode': selectAccessMode})

display(hbox)
display(interactive_output)
display(output)

Output()

Output()